In [2]:
from sqlalchemy import select, func, and_, or_, Integer
from sqlalchemy.orm import selectinload

from models import WorkersOrm, ResumesOrm, Workload
from schemas import ResumesDTO, ResumesRelDTO, WorkersDTO, WorkersRelDTO
from database import session_factory, sync_engine

from pydantic import BaseModel

**Без relationship**

In [4]:
with session_factory() as session:
    query = (
        select(WorkersOrm)
        .limit(2)
    )

    res = session.execute(query)
    result_orm = res.scalars().all()
    print(f"{result_orm=}")
    result_dto = [WorkersDTO.model_validate(row, from_attributes=True) for row in result_orm]
    print(f"{result_dto=}")

2024-11-10 12:42:49,571 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-11-10 12:42:49,572 INFO sqlalchemy.engine.Engine SELECT workers.id, workers.username 
FROM workers 
 LIMIT %(param_1)s::INTEGER
2024-11-10 12:42:49,574 INFO sqlalchemy.engine.Engine [cached since 52.07s ago] {'param_1': 2}
result_orm=[<WorkersOrm id=1, username=Jack>, <WorkersOrm id=2, username=Michael>]
result_dto=[WorkersDTO(username='Jack', id=1), WorkersDTO(username='Michael', id=2)]
2024-11-10 12:42:49,577 INFO sqlalchemy.engine.Engine ROLLBACK


**C relationship**

In [9]:
with session_factory() as session:
    query = (
        select(WorkersOrm)
        .options(selectinload(WorkersOrm.resumes))
        .limit(2)
    )

    res = session.execute(query)
    result_orm = res.scalars().all()
    print(f"{result_orm=}")
    result_dto = [WorkersRelDTO.model_validate(row, from_attributes=True) for row in result_orm]
    print(f"{result_dto=}")

2024-11-10 12:48:14,469 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-11-10 12:48:14,472 INFO sqlalchemy.engine.Engine SELECT workers.id, workers.username 
FROM workers 
 LIMIT %(param_1)s::INTEGER
2024-11-10 12:48:14,474 INFO sqlalchemy.engine.Engine [cached since 84.15s ago] {'param_1': 2}
2024-11-10 12:48:14,478 INFO sqlalchemy.engine.Engine SELECT resumes.worker_id AS resumes_worker_id, resumes.id AS resumes_id, resumes.title AS resumes_title, resumes.compensation AS resumes_compensation, resumes.workload AS resumes_workload, resumes.created_at AS resumes_created_at, resumes.updated_at AS resumes_updated_at 
FROM resumes 
WHERE resumes.worker_id IN (%(primary_keys_1)s::INTEGER, %(primary_keys_2)s::INTEGER)
2024-11-10 12:48:14,479 INFO sqlalchemy.engine.Engine [cached since 84.13s ago] {'primary_keys_1': 1, 'primary_keys_2': 2}
result_orm=[<WorkersOrm id=1, username=Jack>, <WorkersOrm id=2, username=Michael>]
result_dto=[WorkersRelDTO(username='Jack', id=1, resumes=[ResumesDTO

**JOIN**

In [10]:
class WorkloadAvgCompensationDTO(BaseModel):
    workload: Workload
    avg_compensation: int

In [12]:
with session_factory() as session:
    query = (
        select(
            ResumesOrm.workload,
            func.avg(ResumesOrm.compensation).cast(Integer).label("avg_compensation"),
        )
        .select_from(ResumesOrm)
        .filter(and_(
            ResumesOrm.title.contains("Python"),
            ResumesOrm.compensation > 40000,
        ))
        .group_by(ResumesOrm.workload)
        .having(func.avg(ResumesOrm.compensation) > 70000)
    )
    res = session.execute(query)
    result_orm = res.all()
    print(f"{result_orm=}")
    result_dto = [WorkloadAvgCompensationDTO.model_validate(row, from_attributes=True) for row in result_orm]
    print(f"{result_dto=}")

2024-11-10 12:50:43,566 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-11-10 12:50:43,569 INFO sqlalchemy.engine.Engine SELECT resumes.workload, CAST(avg(resumes.compensation) AS INTEGER) AS avg_compensation 
FROM resumes 
WHERE (resumes.title LIKE '%%' || %(title_1)s::VARCHAR || '%%') AND resumes.compensation > %(compensation_1)s::INTEGER GROUP BY resumes.workload 
HAVING avg(resumes.compensation) > %(avg_1)s::INTEGER
2024-11-10 12:50:43,570 INFO sqlalchemy.engine.Engine [cached since 62.94s ago] {'title_1': 'Python', 'compensation_1': 40000, 'avg_1': 70000}
result_orm=[(<Workload.parttime: 'parttime'>, 165000), (<Workload.fulltime: 'fulltime'>, 90000)]
result_dto=[WorkloadAvgCompensationDTO(workload=<Workload.parttime: 'parttime'>, avg_compensation=165000), WorkloadAvgCompensationDTO(workload=<Workload.fulltime: 'fulltime'>, avg_compensation=90000)]
2024-11-10 12:50:43,578 INFO sqlalchemy.engine.Engine ROLLBACK
